# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

## Обзор данных, общая информация

In [146]:
import pandas as pd
from pymystem3 import Mystem
from collections import Counter

In [147]:
data = pd.read_csv('/datasets/data.csv')
m = Mystem()

In [148]:
print(data.info()) # информация о датасете
data.sample(10) # 10 рандомных строк датасета

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB
None


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
1827,0,-356.522646,47,среднее,1,женат / замужем,0,F,компаньон,0,177734.706698,операции с недвижимостью
10293,0,339943.729642,62,высшее,0,вдовец / вдова,2,F,пенсионер,0,249382.417411,операции с недвижимостью
19012,0,-275.504604,38,среднее,1,Не женат / не замужем,4,M,сотрудник,0,109660.236035,операции с коммерческой недвижимостью
18738,0,-164.393584,49,среднее,1,гражданский брак,1,F,сотрудник,0,243056.921615,сыграть свадьбу
11947,1,-202.363830,30,среднее,1,женат / замужем,0,M,сотрудник,0,123639.079405,покупка недвижимости
13306,1,-2122.800089,31,среднее,1,гражданский брак,1,F,сотрудник,1,195738.592716,на проведение свадьбы
10358,0,-798.366064,22,неоконченное высшее,2,гражданский брак,1,F,сотрудник,0,115899.317940,на проведение свадьбы
18516,0,-2615.148409,39,высшее,0,женат / замужем,0,M,сотрудник,0,417683.065179,операции с коммерческой недвижимостью
2929,1,-2306.171209,31,высшее,0,женат / замужем,0,F,госслужащий,0,86574.378855,недвижимость
11107,2,-5437.303642,41,высшее,0,гражданский брак,1,F,сотрудник,0,167287.660324,сыграть свадьбу


In [149]:
data.isnull().mean() # процент пропусков

children            0.000000
days_employed       0.100999
dob_years           0.000000
education           0.000000
education_id        0.000000
family_status       0.000000
family_status_id    0.000000
gender              0.000000
income_type         0.000000
debt                0.000000
total_income        0.100999
purpose             0.000000
dtype: float64

**Вывод**

Исходный датасет размерностью 21525 строк, 12 столбцов.Из общего массива выбиваются 2 столбца (days_employed - общий трудовой стаж, total_income - ежемесячный доход), с количеством значений 19351, явно имеются пропуски, это около 10% данных.

## Предобработка данных

Приведём столбцы с тектовыми значениями к нижнему регистру.

In [150]:
for col in data[['education', 'family_status', 'income_type']].columns:
    data[col] = data[col].str.lower()

### Обработка пропусков

Оценим количество пропусков в столбце общего стажа по типу клиентов

In [151]:
print('Пропуски по типу занятости:')
print(data[data['days_employed'].isnull()]['income_type'].value_counts())

print('\nКоличество записей относящихся к разным типам занятости:')
print(data['income_type'].value_counts())

Пропуски по типу занятости:
сотрудник          1105
компаньон           508
пенсионер           413
госслужащий         147
предприниматель       1
Name: income_type, dtype: int64

Количество записей относящихся к разным типам занятости:
сотрудник          11119
компаньон           5085
пенсионер           3856
госслужащий         1459
безработный            2
предприниматель        2
студент                1
в декрете              1
Name: income_type, dtype: int64


Пропуски имеются практически во всех категориях занятости.

Заполним пропуски медианными значениями в столбцах 'days_employed' и 'total_income' исходя из типа занятости клиента

In [152]:
data.loc[data['total_income'].isnull(),'total_income'] = data.groupby('income_type')['total_income'].transform('median')
data.loc[data['days_employed'].isnull(), 'days_employed'] = data.groupby('income_type')['days_employed'].transform('median')

Рассмотрим возраст заемщиков

In [153]:
print(data['dob_years'].describe())
print('\nКлиенты с указанным возрастом меньше 18 лет')
data[data['dob_years'] < 18]['dob_years'].value_counts() # клиенты с указанным возрастом меньше 18 лет.

count    21525.000000
mean        43.293380
std         12.574584
min          0.000000
25%         33.000000
50%         42.000000
75%         53.000000
max         75.000000
Name: dob_years, dtype: float64

Клиенты с указанным возрастом меньше 18 лет


0    101
Name: dob_years, dtype: int64

Выявлен неявный пропуск, возраст 101 заёмщика равен 0. Необходимо запонить данные значения.

In [154]:
data['dob_years'] = data.groupby('income_type')['dob_years'].transform('median')
data[data['dob_years'] == 0]
data.sample(20)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
10443,0,-768.902569,39.0,высшее,0,гражданский брак,1,M,сотрудник,1,177817.682654,на проведение свадьбы
861,1,-1304.662716,39.0,неоконченное высшее,2,женат / замужем,0,M,сотрудник,0,89553.530138,операции с жильем
9868,0,-4611.244402,39.0,среднее,1,женат / замужем,0,F,компаньон,0,283535.145353,операции со своей недвижимостью
7521,1,-1891.062856,39.0,высшее,0,женат / замужем,0,M,компаньон,0,218487.793621,покупка своего жилья
1815,1,-247.599668,39.0,среднее,1,гражданский брак,1,F,сотрудник,0,114077.518783,на проведение свадьбы
12161,3,-853.460285,39.0,высшее,0,женат / замужем,0,M,сотрудник,1,166571.732985,заняться образованием
4338,0,-1515.835530,39.0,среднее,1,женат / замужем,0,M,сотрудник,0,136818.115423,покупка своего жилья
9859,0,-5003.783985,39.0,среднее,1,женат / замужем,0,F,сотрудник,0,393814.674506,образование
6077,1,-1547.382223,39.0,высшее,0,женат / замужем,0,F,компаньон,0,172357.950966,получение высшего образования
7861,0,371118.132244,60.0,среднее,1,гражданский брак,1,F,пенсионер,0,74472.707322,на проведение свадьбы


In [155]:
data['children'].value_counts()

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

47 клиентов имеют отрицательное количество детей, скорее всего это опечатка. Возьмем абсолютное значение.
Значение 20 оставим без изменений, вдруг у семьи не только свои, но и приёмные дети.

In [156]:
data.loc[data['children'] == -1,'children'] = abs(data.loc[data['children'] == -1,'children'])
data['children'].value_counts()

0     14149
1      4865
2      2055
3       330
20       76
4        41
5         9
Name: children, dtype: int64

**Вывод**

В столбцах days_employed - общий трудовой стаж в днях, total_income - ежемесячный доход - 2174 пропуска в каждом. Это 10% значений, пренебрегать этими данными не можем, поэтому запоним пропуски, медианными значениями от категории клиентов.

Отсутствующие данные по общему трудовому стажу и ежемесячному доходу имеют значия типа NaN. Данные в столбцах могут отсутствовать у клиентов, если заемная сумма позволяет взять кредит без этих данных. При сортировке отсутствующих данных по типу занятости можно выделить: сотрудник, компаньон, пенсионер, госслужащий. По сотрудникам и компаньонам, у банка скорее всего данные находятся в другой базе и при необходимости таблицы можно смержить. Пенсионеры и госслужащие получают бюджетные денежные средства, поэтому эти данные при желании можно получить у ФСС.

Для исключения одинаковых строк с разным регистром, приведём таблицу к нижнему регистру.

Возраст некоторых участников = 0, заполним так же медианными значениями, с поправкой на тип занятости.

### Замена типа данных

Для последующего анализа приведем столбцы days_employed - общий трудовой стаж и total_income доход к целочисленному типу.

In [157]:
data['days_employed'].isnull().sum()

0

In [158]:
data['days_employed'] = data['days_employed'].astype('int')
data['total_income'] = data['total_income'].astype('int')
data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437,39.0,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья
1,1,-4024,39.0,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,-5623,39.0,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья
3,3,-4124,39.0,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование
4,0,340266,60.0,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу


### Обработка дубликатов

In [159]:
print('Количество полных дубликатов =', data.duplicated().sum(),'/',(data.duplicated().sum()/data.shape[0]).round(2))
print(data.info())
data = data.drop_duplicates().reset_index(drop=True)
print()
print(data.info())

Количество полных дубликатов = 768 / 0.04
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null int64
dob_years           21525 non-null float64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null int64
purpose             21525 non-null object
dtypes: float64(1), int64(6), object(5)
memory usage: 2.0+ MB
None

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20757 entries, 0 to 20756
Data columns (total 12 columns):
children            20757 non-null int64
days_employed       20757 non-null int64
dob_years           20757 non-null float64
education           20757 non-null object
educat

Вывод

Обнаружено 768 полных дубликата, 4% от начального объема. Дубликаты есть в самых численных категориях: сотрудник, компаньон, пенсионер, госслужащий. Появление дубликатов возможно при слиянии таблиц, когда одни и те же люди находятся под разными ID. Возможно это созаемщики, которых записали с параметрами заемщика.

### Лемматизация

In [160]:
lemas = []
data['purpose'].unique() # посмотрим какие уникальные значения имеются в столбце purpose
for text in data['purpose'].unique():
    lemas += m.lemmatize(text)
    
Counter(lemas) #относительно уникальных значений посмотрим как часто встречаются те или иные леммы


 
# на основе лемм найденных выше создадим словарь с типизированными целями
dict = {
    'образование':'образование', 'автомобиль':'автомобиль',
    'недвижимость':'недвижимость', 'образование':'образование',
    'свадьба':'свадьба', 'жилье':'недвижимость'
}

def func(row): # функция возвращающая подходящую типизированную цель исходя из значений лемм в столбце purpose
    for query in m.lemmatize(row['purpose']):
        if query in dict:
             return dict[query]

data['purpose_group'] = data.apply(func, axis=1) # создаём новый столбец типизированных целей кредита


data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_group
0,1,-8437,39.0,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,недвижимость
1,1,-4024,39.0,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,автомобиль
2,0,-5623,39.0,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,недвижимость
3,3,-4124,39.0,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,образование
4,0,340266,60.0,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,свадьба


**Вывод**

Проведём лемматизацию по столбцу целей кредита "purpose", выделим 5 основных категорий, для удобства категоризации данных, создадим с ними словарь. Создадим новый столбец, чтобы не изменять основной структуры таблицы. Выделим следующие целевые типы: образование, автомобиль, недвижимость, образование, свадьба.

### Категоризация данных

In [161]:
def dob_years_group (row):
    if (row['dob_years'] >= 18) & (row['dob_years'] < 30): return '18-29'
    elif (row['dob_years'] >= 30) & (row['dob_years'] < 40): return '30-39'
    elif (row['dob_years'] >= 40) & (row['dob_years'] < 50): return '40-49'
    elif (row['dob_years'] >= 50) & (row['dob_years'] < 60): return '50-59'
    elif row['dob_years'] >= 60: return '60+'
def total_income_group (row):
    if row['total_income'] < 100000: return '-100'
    elif (row['total_income'] >= 100000) & (row['total_income'] < 200000): return '100-200'
    elif (row['total_income'] >= 200000) & (row['total_income'] < 300000): return '200-300'
    elif row['total_income'] >= 300000: return '300+'
    
def children_group (row):
    if row['children'] == 0: return '0 детей'
    elif row['children'] == 1: return '1 ребёнок'
    elif row['children'] == 2: return '2 ребёнка'
    elif row['children'] >= 3: return '3 и более детей'
    
data['dob_years_group'] = data.apply(dob_years_group, axis=1)
data['total_income_group'] = data.apply(total_income_group, axis=1)
data['children_group'] = data.apply(children_group, axis=1)

data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_group,dob_years_group,total_income_group,children_group
0,1,-8437,39.0,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,недвижимость,30-39,200-300,1 ребёнок
1,1,-4024,39.0,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,автомобиль,30-39,100-200,1 ребёнок
2,0,-5623,39.0,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,недвижимость,30-39,100-200,0 детей
3,3,-4124,39.0,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,образование,30-39,200-300,3 и более детей
4,0,340266,60.0,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,свадьба,60+,100-200,0 детей


**Вывод**

Категории по детям: (создадим отдельный столбец для обобщения)
- нет детей;
- 1 ребёнок;
- 2 ребёнка;
- 3 и более.

Категории по возрасту: (создадим отдельный столбец для обобщения)
- 18 - 30;
- 30 - 40;
- 40 - 50;
- 50 - 60;
- 60 и более.

Заработок разобьем на группы: (создадим отдельный столбец для обобщения)
- до 100 000 включительно;
- от 100 000 до 200 000 включительно;
- от 200 000 до 300 000 включительно;
- от 300 000 и более.

## Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [162]:
pivot_children_debt = pd.pivot_table(data, index=['children_group'], columns='debt', values = 'purpose_group', aggfunc='count')
pivot_children_debt

debt,0,1
children_group,,
0 детей,12500,1051
1 ребёнок,4293,444
2 ребёнка,1820,194
3 и более детей,416,39


In [163]:
pivot_children_debt['0%'] = round(pivot_children_debt[0] / (pivot_children_debt[0] + pivot_children_debt[1]) * 100)
pivot_children_debt['1%'] = round(pivot_children_debt[1] / (pivot_children_debt[0] + pivot_children_debt[1]) * 100)

In [164]:
pivot_children_debt = pivot_children_debt.astype('int')
pivot_children_debt.set_axis(['Return_count', 'UN_Return_count', 'Return_%', 'UN_Return_%'], axis='columns', inplace=True)
pivot_children_debt

,Return_count,UN_Return_count,Return_%,UN_Return_%
children_group,,,,
0 детей,12500,1051,92,8
1 ребёнок,4293,444,91,9
2 ребёнка,1820,194,90,10
3 и более детей,416,39,91,9


**Вывод**

Приведём таблицу к целочисленному типу и переименуем столбцы для наглядности.

Вывод: Заёмщики без детей справляются лучше всех. Заёмщики с детьми справляются хуже, вне зависимости от их количества.

Возможно возврат по кредиту более выражено зависит от других параметров. Рассмотрим их.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [165]:
pivot_family_debt = pd.pivot_table(data, index=['family_status'], columns='debt', values = 'purpose_group', aggfunc='count')
pivot_family_debt

debt,0,1
family_status,,
в разводе,1097,85
вдовец / вдова,877,63
гражданский брак,3624,381
женат / замужем,10944,925
не женат / не замужем,2487,274


In [166]:
pivot_family_debt['0%'] = round(pivot_family_debt[0] / (pivot_family_debt[0] + pivot_family_debt[1]) * 100)
pivot_family_debt['1%'] = round(pivot_family_debt[1] / (pivot_family_debt[0] + pivot_family_debt[1]) * 100)
pivot_family_debt.set_axis(['Return_count', 'UN_Return_count', 'Return_%', 'UN_Return_%'], axis='columns', inplace=True)
pivot_family_debt = pivot_family_debt.astype('int')
pivot_family_debt

,Return_count,UN_Return_count,Return_%,UN_Return_%
family_status,,,,
в разводе,1097,85,93,7
вдовец / вдова,877,63,93,7
гражданский брак,3624,381,90,10
женат / замужем,10944,925,92,8
не женат / не замужем,2487,274,90,10


**Вывод**

Так же как и в прошлой сводной таблице добавим 2 столбца с процентами для наглядности.

Вывод по таблице: Лучше всех исполняют свои долговые обязательства люди потерявшие своих супругов по той или иной причине. Хуже всех погашают кредиты заемщики указавшие своё семейное положение, как не женат / не замужем.

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [167]:
pivot_income = pd.pivot_table(data, index=['total_income_group'], columns='debt', values = 'purpose_group', aggfunc='count')
pivot_income.head(20)

debt,0,1
total_income_group,,
-100,4109,354
100-200,10211,1016
200-300,3332,252
300+,1377,106


In [168]:
pivot_income['0%'] = round(pivot_income[0] / (pivot_income[0] + pivot_income[1]) * 100)
pivot_income['1%'] = round(pivot_income[1] / (pivot_income[0] + pivot_income[1]) * 100)
pivot_income.set_axis(['Return_count', 'UN_Return_count', 'Return_%', 'UN_Return_%'], axis='columns', inplace=True)
pivot_income = pivot_income.astype('int')
pivot_income

,Return_count,UN_Return_count,Return_%,UN_Return_%
total_income_group,,,,
-100,4109,354,92,8
100-200,10211,1016,91,9
200-300,3332,252,93,7
300+,1377,106,93,7


**Вывод**

Среди заёмщиков по уровню дохода, вовремя рассчитываются с банком заёмщики с высоким уровнем дохода. Группы: 200-300 и 300+. Но несмотря на низкий доход (группа -100), заёмщики этой группы рассчитываются с банком лучше, чем группа (100 - 200))

- Как разные цели кредита влияют на его возврат в срок?

In [169]:
data_purpose_debt = pd.pivot_table(data, index=['purpose_group'], columns='debt', values = 'purpose', aggfunc='count')
data_purpose_debt.head(20)

debt,0,1
purpose_group,,
автомобиль,3788,401
недвижимость,9680,780
образование,3543,368
свадьба,2018,179


In [170]:
data_purpose_debt['0%'] = round(data_purpose_debt[0] / (data_purpose_debt[0] + data_purpose_debt[1]) * 100)
data_purpose_debt['1%'] = round(data_purpose_debt[1] / (data_purpose_debt[0] + data_purpose_debt[1]) * 100)
data_purpose_debt.set_axis(['Return_count', 'UN_Return_count', 'Return_%', 'UN_Return_%'], axis='columns', inplace=True)
data_purpose_debt = data_purpose_debt.astype('int')
data_purpose_debt

,Return_count,UN_Return_count,Return_%,UN_Return_%
purpose_group,,,,
автомобиль,3788,401,90,10
недвижимость,9680,780,93,7
образование,3543,368,91,9
свадьба,2018,179,92,8


**Вывод**

Явный лидер по погашению, это заёмщики берущие денежные средства на недвижимость. За образование или автомобиль рассчитываются менее охотно.

## Общий вывод

Анализ показал, что лучше расплачиваются люди без детей, в разводе или вдовствующие, с доходом более 200000, оформляющие кредит на недвижимость.